# Klargjøre og laste grunnlagsdata til Statistikkbanken

Så langt kan alle seksjoner i SSB publisere til Statistikkbanken ved hjelp av SAS. Nå er det laget en Python funksjon som gjør at vi nå også kan publisere til Statistikkbanken ved hjelp av Python i stedet. I tillegg skal R funksjonen også være operativ igjen. Så SSB har nå muligheten for å publisere til Statistikkbanken i alle programmeringsspråkene våres: SAS, R, og Python.

Dette gjør at alle seksjoner i SSB har muligheten til å skrive om lasteprogrammene sine fra SAS til Python som en del av omleggingsfasen til Dapla, samtidig å kunne ta det i bruk i dagens produksjon. Kanskje er det god læring for seksjonene for å bygge kompetanse i Python programmering ved å gå løs på denne type oppgave!

Jeg kommer her til å demonstrere hvordan du kan lage datagrunnlaget for en Statistikk vi har i Statistikkbanken, Eksport av Laks, og laste opp data for publisering til Statistikkbanken.

## Importere moduler

In [ ]:
import numpy as np
import pandas as pd
import getpass
import cx_Oracle
import os
import shutil

In [ ]:
# Hvor står vi nå?
os.getcwd()

## .dat-filen, hvordan skal den se ut?

Henter opp og kopierer over en .dat fil som min seksjon har liggende. Kopierer over til mitt arbeidsområde så vi kan se nærmere på hvordan filen ser ut.

In [ ]:
datfil = "ekslaksuke1.dat"
felles = os.getenv('UTENRIKS') + r"/c_statbank/wk24"
src_path = felles + os.sep + datfil

navn = "ekslaksuke1test.dat"
hjemme = os.getcwd() + os.sep
dst_path = hjemme + navn

print(src_path)
print(dst_path)

In [ ]:
# kopiere filer over til pathen
shutil.copyfile(src_path, dst_path)

Laster opp dataene fra fila så vi får se nærmere på hvordan det skal se ut. Dette lager hele grunnlaget for hvordan vi må sette opp dataene våres...

In [ ]:
test1laks = pd.read_csv(navn, sep=';', header=None)
test1laks

In [ ]:
test1laks.dtypes

## Samle inn - Hente grunnlags data fra Oracle databasen

Setter opp connection til Oracle databasen. Du blir spurt om Oracle passord her.

In [ ]:
conn = cx_Oracle.connect(getpass.getuser() + "/" + getpass.getpass(prompt='Angi oracle-passord for bruker ' + getpass.getuser() + ':')+"@DB1P")

Setter opp sql spørringen som vi skal sende til Oracle databasen.

In [ ]:
query = """\
SELECT IMPEKS, EKSNR, AAR, LOPENR, LINJENR, UKENR, VARENR, VERDI, VEKT \
FROM UTENRIKSHANDEL.UTH_VARELINJER \
WHERE IMPEKS='2' AND UKENR='202224' \
AND VARENR IN ('03021411','03021419','03031311','03031319')\
"""

Ser om sql spørringen ser fin ut.

In [ ]:
query

Henter dataene fra Oracle databasen, som legges direkte inn i en Dataframe.

In [ ]:
df = pd.read_sql_query(query, conn)

df

Lukker connection til Oracle Databasen.

In [ ]:
conn.close()

## Klargjøring - Transformering av data

Jeg har mottatt alle kolonne navnene med store bokstaver, som lager litt ekstra arbeid, så legger disse om til små bokstaver.

In [ ]:
df.columns = df.columns.str.lower()
df.dtypes

Lager ny variabel type som skal ha to mulige verdier avhengig av verdien i kolonnen varenr. Dette er da en where statement, og fungerer helt likt som i Excel vel og merke... Variabelen type er noe Statistikkbanken forventer, som viste seg da vi leste inn .dat filen første gang.

In [ ]:
df['type'] = np.where(df.varenr == '03021411', '01', np.where(df.varenr == '03021419', '01', '02'))
df

Summerer opp til aggregatene type og ukenr. De kolonnene som enten er Float eller Int vil bli summert opp.

In [ ]:
laks = df.groupby(['type', 'ukenr']).sum().reset_index()
laks

Greit å sjekke datatypene i kolonnene løpende i ny og ne...

In [ ]:
laks.dtypes

Justerer verdiene i kolonnen Ukenr så det passer med det Statistikkbanken skal ha.

In [ ]:
laks['ukenr'] = laks.ukenr[0][:4] + 'U' + laks.ukenr[0][-2:]
laks

Lager prisen per kilo for laksen. Må ha 2 desimaler.

In [ ]:
laks['pkg'] = laks.verdi.div(laks.vekt).round(2)
laks

Gjør om verdiene i kolonnen vekt fra kilo til tonn. Skal ikke ha noen desimaler.

In [ ]:
laks.vekt = laks.vekt.div(1000).round(0).astype(int)
laks

Da trenger vi ikke variabelen verdi lenger, så da kan vi slette denne.

In [ ]:
del laks["verdi"]
laks

Pandas opererer med punktum i stede for komma, og Statistikkbanken forventer komma. Gjør derfor alle variablene i dataframen om til strings.

In [ ]:
for col in laks.columns:
    laks[col] = laks[col].astype(str)

In [ ]:
laks.dtypes

ok, og så endrer jeg fra punktum til komma.

In [ ]:
laks = laks.stack().str.replace('.',',').unstack()
laks

Og da er dataene slik vi ønsker det!

## Lagre data som .dat-fil

For å lagre dataene i dataframen til en .dat fil kan vi benytte pandas sine metoder for hvordan den prosesserer csv filer. Dataene i .dat fila skilles kun med semikolon. Skal også ikke være noen overskrifts rad, og indexen fra dataframen vil vi ikke ha med inn i filen.

In [ ]:
datfile = "ekslaksuke1.dat"
laks.to_csv(datfile, sep=';', header=False, decimal=',', index=False)

Leser filen inn igjen for å se om vi får det samme resultatet med .dat filen vi hadde fra før og den vi nettopp lagde.

In [ ]:
test1laks = pd.read_csv(datfile, sep=';', header=None)
test1laks

In [ ]:
test1laks.dtypes

In [ ]:
test2laks = pd.read_csv(navn, sep=';', header=None)
test2laks

In [ ]:
test2laks.dtypes

OK, da er vi klare for å laste opp denne filen til Statistikkbanken!

## Publisere til Statistikkbanken

En funksjon er lagd og lagt på felles området på Linux i SSB som vi kan benytte. Det er også lagt opp til at Python leter etter pakker/moduler/biblioteker i denne fellesmappen også, slik at vi kan importere den som en eller vanlig modul.

In [ ]:
from statbankpy import statbank_lasting

For nærmere beskrivelse av funksjonen:

In [ ]:
help(statbank_lasting)

I dette tilfelle må vi sende inn 5 inputs til funksjonen. Filnavn og filsti har vi fra før, men vi må også sende inn publiseringsdatoen, lastebrukeren for seksjonen det gjelder, og tabellnavn som filene skal lastes opp til.

In [ ]:
pubdato = "20220628" #OBS! skriv i formatet YYYYMMDD, funksjonen vil rette opp deretter til korrekt format.
laste_bruker = "LAST214"
lakstab = "EksLaksUke"

print("Publinseringsdato som er satt: " + pubdato)
print("Lastebruker: " + laste_bruker)
print("Tabell vi skal laste opp til: " + lakstab)
print("Filstien hvor .dat-fila ligger: " + hjemme)
print("Filnavnet til .dat-fila: " + datfile)

In [ ]:
statbank_lasting(pdato=pubdato,
                 luser=laste_bruker,
                 tab=lakstab,
                 filsti=hjemme,
                 fil=datfile)

Det vil bli printet ut løpende fra funksjonen om hvordan lastingen gikk, slik de fleste i SSB er vant med å få fra SAS.